In [0]:
import os

import pandas as pd

!rm -rf nem-data
!git clone https://github.com/ADGEfficiency/nem-data
%cd nem-data
!python setup.py install
!pip3 install -q -r requirements.txt
!nem --reports demand --start 2018-01 --end 2018-03

# Data cleaning

Before using this notebook, you will need to have used the `nem-data` library to download data - see the instructions for this here

Cleaning data requires two tasks
- identification
- cleaning

See what files you have downloaded:

In [0]:
home = os.path.expanduser('~')
nem_files = os.listdir(os.path.join(home, 'nem-data', 'demand'))

nem_files

If we look at a specific month, we see that `nem-data` has downloaded a few files for each month:

In [0]:
nem_files = os.listdir(os.path.join(home, 'nem-data', 'demand', '2018-01'))

nem_files

Let's grab `clean.csv` and extract a potential target.

The data provided for the NEM is large - making working on problems in the NEM great for data scientists:

In [0]:
raw = pd.read_csv(os.path.join(home, 'nem-data', 'demand', '2018-01', 'clean.csv'), index_col=0)
raw.head()

In [0]:
region = 'SA1'
mask = raw.loc[:, 'REGIONID'] == region
target = raw.loc[mask, ['SETTLEMENTDATE', 'REGIONID', 'TOTALDEMAND']]

target.head()

## Exercise - raw dataset health check

On the `raw` dataframe, check
- how many missing values we have in each column
- check for duplicates

On the `target` dataframe
- check the integrity of the time stamps (do we have any gaps?)